In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/finaltrainbytookk/TTS

/content/drive/MyDrive/finaltrainbytookk/TTS


In [ ]:
!make system-deps  # only on Linux systems.
!make install

sudo apt-get install -y libsndfile1-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.31-2build1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
pip install -e .[all]
Obtaining file:///content/drive/MyDrive/finaltrainbytookk/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached Cython-0.29.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#for new training
#!CUDA_VISIBLE_DEVICES=0,1,2 python /content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/train_vits_tts.py

In [ ]:
#for continue training
#!CUDA_VISIBLE_DEVICES=0,1,2 python /content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/train_vits_tts.py --continue_path /content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/vits_vctk-July-20-2023_07+50AM-0000000

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=ee821fa4ffeffb91e30cd328b8dd34934dcc181a5ce574183a928f83e3915483
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.5 MB/s eta 0:00:00
    

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!ngrok authtoken 2Sy2LWPQ2xqvJOk65EY8RMwsEKA_6NfruRjiBmV7s6BFhkUap
!pip install speechrecognition
!pip install gradio TTS
from flask import Flask, request
import speech_recognition as sr

app = Flask(__name__)

class Transcribe_File:
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def transcribe_audio(self, audio_file):
        with sr.AudioFile(audio_file) as source:
            audio_data = self.recognizer.record(source)
            try:
                text = self.recognizer.recognize_google(audio_data, language="lo-LA")
                return {"transcription": text}
            except sr.UnknownValueError:
                return {"error": "Google Speech Recognition could not understand the audio"}
            except sr.RequestError as e:
                return {"error": f"Could not request results from Google Speech Recognition service; {e}"}



In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from distutils.log import error
import mimetypes
from re import template
from urllib import request
import io
import os
from flask import render_template, request, send_file
import tempfile
import gradio as gr
import numpy as np
import os
import json
from typing import Optional
from TTS.config import load_config
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer

model_path = "/content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/vits_vctk-July-20-2023_07+50AM-0000000/checkpoint_100000.pth" #path of model_path
config_path = "/content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/vits_vctk-July-20-2023_07+50AM-0000000/config.json" #path pf config_path
speakers_file_path = "/content/drive/MyDrive/finaltrainbytookk/TTS/recipes/multilingual/vits_tts/vits_vctk-July-20-2023_07+50AM-0000000/speakers.pth" #path of speakers_file_path

speaker_idx = "Nick"

synthesizer = Synthesizer(
    model_path, config_path, speakers_file_path, None, None, None,
)
PEOPLE_FOLDER = os.path.join('content/drive/MyDrive/website_with_flask', 'templates')
app = Flask(__name__, template_folder="/content/drive/MyDrive/website_with_flask/templates", static_folder="/content/drive/MyDrive/website_with_flask/templates")
app.config['UPLOAD_FOLDER'] = PEOPLE_FOLDER
run_with_ngrok(app)
@app.route("/")

def index():
    full_filename = os.path.join(app.config['UPLOAD_FOLDER'], 'LoiyToiyDev.png')
    return render_template("index.html",user_image = full_filename)

@app.route("/call/martha", methods = ["POST"])
def call_martha():
    if not request.form["text"]:
        return {"error": "Please provide the text"}, 400
    text = request.form["text"]
    outputs = synthesizer.tts(text, speaker_idx,language_name="lo-la")
    out = io.BytesIO()
    synthesizer.save_wav(outputs, out)
    return send_file(out, mimetype="audio/wav")

@app.route("/transcribe", methods=["POST"])
def transcribe():
    transcriber = Transcribe_File()
    audio_file = request.files["audio_file"]
    if audio_file:
        result = transcriber.transcribe_audio(audio_file)
        return result
    else:
        return {"error": "No audio file provided"}
app.run()

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://467b-34-23-48-173.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 06:03:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 06:03:06] "GET /templates/FSN_Original.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 06:03:06] "GET /templates/LoiyToiyDev.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 06:03:08] "GET /templates/text-to-speech.png HTTP/1.1" 200 -


 > Text splitted to sentences.
['ດາວໜ້າຮັກທີ່ສຸດ']


...

INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 08:08:38] "POST /call/martha HTTP/1.1" 200 -


 > Processing time: 5.033872127532959
 > Real-time factor: 0.5198430142942195
 > Text splitted to sentences.
['ບົດໂຄງການຈົບຊັ້ນລະດັບປະລິນຍາຕີວິທະຍາສາດ ການຮຽນຮູ້ ແລະ ການຈື່ຈຳ ຂໍ້ຄວາມຂຽນພາສາລາວໄປເປັນຂໍ້ຄວາມສຽງ ດ້ວຍເຕັກນິກຂອງປັນຍາປະດິດຂັ້ນສູງ', 'ກະລຸນາເລືອກຟາຍສຽງ']


INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 08:09:35] "POST /call/martha HTTP/1.1" 200 -


 > Processing time: 5.714981317520142
 > Real-time factor: 0.47785211917288223


INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 08:11:13] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Aug/2023 08:11:54] "POST /transcribe HTTP/1.1" 200 -
